https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download

In [3]:
from getpass import getpass
token = getpass('GitHub token: ')

GitHub token: ··········


In [4]:
!git clone --branch main https://$token@github.com/Bartekb3/fromscratch-transformer-classifier.git

Cloning into 'fromscratch-transformer-classifier'...
remote: Enumerating objects: 1357, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 1357 (delta 182), reused 243 (delta 130), pack-reused 1006 (from 1)
Receiving objects: 100% (1357/1357), 20.79 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (680/680), done.


In [5]:
%cd /content/fromscratch-transformer-classifier

/content/fromscratch-transformer-classifier


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp -r ../drive/MyDrive/tokenizers/* ./src/textclf_transformer/tokenizer/

In [8]:
from sklearn.model_selection import train_test_split
import os
import torch
import sys
from pathlib import Path

In [9]:
import pandas as pd
from src.textclf_transformer import *

In [10]:
from datasets import load_dataset, concatenate_datasets

ds = load_dataset("imdb")

merged = concatenate_datasets([ds["test"], ds["train"]])
df = pd.DataFrame(merged)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
df['text']

,text
0,I love sci-fi and am willing to put up with a ...
1,"Worth the entertainment value of a rental, esp..."
2,its a totally average film with a few semi-alr...
3,STAR RATING: ***** Saturday Night **** Friday ...
4,"First off let me say, If you haven't enjoyed a..."
...,...
49995,A hit at the time but now better categorised a...
49996,I love this movie like no other. Another time ...
49997,This film and it's sequel Barry Mckenzie holds...
49998,'The Adventures Of Barry McKenzie' started lif...


In [12]:
df.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [13]:
df["text"] = df["text"].str.replace(r"<br\s*/?>", "", regex=True)

In [14]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label'])

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 40000 | Val: 5000 | Test: 5000


In [16]:
tok = WordPieceTokenizerWrapper()

In [23]:
tokenizer_name = 'IMDB_tokenizer_10'

In [24]:
tok.load(tokenizer_dir= f'./src/textclf_transformer/tokenizer/{tokenizer_name}')

In [25]:
tokens_train = tok.encode_pandas(train_df, 'text',
                                  max_length=512,
                                  label_col='label'
                                 )
tokens_val = tok.encode_pandas(val_df, 'text',
                                max_length=512,
                                label_col='label')
tokens_test = tok.encode_pandas(test_df, 'text',
                                 max_length=512,
                                 label_col='label')

In [26]:
print(tok.tokenizer.decode(tokens_train[0][0]))
print(tokens_train[0][0])  # label

[CLS] i ' d love to give kolchak a higher rating but the show quickly went from scary / suspenseful to silly. abc ' s fault. they moved the show to friday nights at 8 : 00 p. m., then known as the " family hour ". never should have been on fridays in the first place. i was a sophomore in high school and loved the early episodes! it was first up against police woman on nbc. abc had huge problems with friday nights. bad season for them overall until barney miller, baretta, and swat debuted in january of ' 75. kolchak should have been a hit. darren mcgavin begged to get out of his contract to end the show. too bad the writing wasn ' t up to richard matheson ' s in the original tv movies. still, mcgavin made kolchak his own, as actors can do. jackie gleason as ralph kramden and caroll o ' connor as archie bunker come to mind. that ins set with the manual typewriters and clacking teletypes seems quaint and ancient today, yet that was part of the appeal. they were very lucky to have simon oa

In [27]:
save_dir = Path("./data/tokenized")
save_dir.mkdir(parents=True, exist_ok=True)

torch.save(tokens_train, save_dir / f"imdb_train_{tokenizer_name}.pt")
torch.save(tokens_val, save_dir / f"imdb_val_{tokenizer_name}.pt")
torch.save(tokens_test, save_dir / f"imdb_test_{tokenizer_name}.pt")

In [28]:
!cp -r ./data/tokenized/imdb_train_{tokenizer_name}.pt ../drive/MyDrive/data/tokenized/
!cp -r ./data/tokenized/imdb_test_{tokenizer_name}.pt ../drive/MyDrive/data/tokenized/
!cp -r ./data/tokenized/imdb_val_{tokenizer_name}.pt ../drive/MyDrive/data/tokenized/